In [3]:
i = 0

In [7]:
i += 1
with tf.variable_scope('debug_%d'%i, reuse=None):
    model = SAaT(n_time_steps=16, 
                 loss_reg_coef=1.0,
                 word_to_idx=test_data['word_to_idx'],
                 wordvec_emb_dim=512,
                 features_dim=[196, 512], 
                 hidden_dim=1024)
    logits = model.build_model()


In [8]:
logits

<tf.Tensor 'debug_3/logits/add_1:0' shape=(?, 23110) dtype=float32>

In [11]:
sess = tf.Session()
tf.global_variables_initializer().run(session=sess)
l = logits.eval(session=sess, feed_dict={model.captions: test_data['captions'],
                                model.features: test_data['features']})

In [14]:
tf.arg_max

355428

In [1]:
import tensorflow as tf
class SAaT(object):
    def __init__(self,
                 n_time_steps, 
                 loss_reg_coef,
                 word_to_idx,
                 wordvec_emb_dim,
                 features_dim, 
                 hidden_dim):
        
        self.n_time_steps = n_time_steps
        self.loss_reg_coef = loss_reg_coef
        
        # word dictionary
        self.word_to_idx = word_to_idx
        self.vocab_size = len(word_to_idx)
        
        # similar to original's dim_emb
        self.wordvec_emb_dim = wordvec_emb_dim
        
        # feature dimensions
        self.features_dim = features_dim
        self.hidden_dim = hidden_dim
        
        # placeholders for captions and features
        self.captions = tf.placeholder(tf.int32, [None, n_time_steps + 1])
        self.features = tf.placeholder(tf.float32, [None, features_dim[0], features_dim[1]])
        
        # initializers
        self.weight_initializer = tf.contrib.layers.xavier_initializer()
        self.bias_initializer = tf.constant_initializer(0.0)

    
    def build_model(self):
        # processing data
        captions = self.captions
        features = self.features
        batch_size = tf.shape(features)[0]
        captions_clipped = self.captions[:, :self.n_time_steps]
        # initial LSTM state
        c,h = self._get_initial_lstm(self.features)

        state = tf.contrib.rnn.LSTMStateTuple(c, h)

        word_embs = self._get_word_embedding(captions_clipped)
        # word_embs = [n_captions, n_time_steps, word_embeddings]

        # build models
        loss = 0.0
        alphas = []
        betas = []
        LSTM = tf.contrib.rnn.LayerNormBasicLSTMCell(self.hidden_dim,
                                                     dropout_keep_prob=0.5,
                                                     dropout_prob_seed=0)
    
        # truncated backprop
        for tstep in range(self.n_time_steps):
            context, alpha, beta = self._get_attention_layer(h, features, reuse=(tstep is not 0))
            # context = [n_features, features_dim[1]]
            # alpha = [n_features, 1, 1]
            # beta = [n_features, 1]
            alphas.append(alpha)
            betas.append(beta)
            
            lstm_input_word = tf.concat([word_embs[:, tstep, :], context], axis=1)
            
            # run the LSTM    
            with tf.variable_scope('lstm', reuse=(tstep is not 0)):
                # each LSTM gate takes input [h_last, context, Ey]
                # the dimension of word vectors = [n_examples, n_words_in_example, word_emb_dim]
                # each step we feed in a word in a sentence
                output, state = LSTM(lstm_input_word, state=state)
                [c, h] = [state.c, state.h]
                
            # LSTM creates an output at each step, hence the loss
            logits = self._get_logits(h, reuse=(tstep is not 0))
            return logits
            loss += self._get_loss(logits=logits, 
                                   labels_gt=captions_clipped[:,tstep], 
                                   alphas=alphas, 
                                   reg_coef=self.loss_reg_coef,
                                   reuse=(tstep is not 0))

        return loss / tf.to_float(batch_size)
                
        
        
    def _get_initial_lstm(self, features):
        # annotation mean
        features_mean = tf.reduce_mean(features, axis=1)

        weight_dim = [self.features_dim[1], self.hidden_dim]
        bias_dim = [self.hidden_dim]
        with tf.variable_scope('lstm_initial'):
            with tf.variable_scope('cell_state'):
                w = tf.get_variable('w', weight_dim, initializer=self.weight_initializer)
                b = tf.get_variable('b', bias_dim, initializer=self.bias_initializer)
                _c = tf.matmul(features_mean, w) + b
                c = tf.nn.tanh(_c)
            with tf.variable_scope('hidden_state'):
                w = tf.get_variable('w', weight_dim, initializer=self.weight_initializer)
                b = tf.get_variable('b', bias_dim, initializer=self.bias_initializer)
                _h = tf.matmul(features_mean, w) + b
                h = tf.nn.tanh(_h) 
            
        return c, h
    
    def _get_word_embedding(self, word_idx, reuse=False):
        word_vecs_dim = len(self.word_to_idx)
        weight_dim = [self.vocab_size, self.wordvec_emb_dim]
        emb_weight_initializer = tf.random_uniform_initializer(minval=-1.0, maxval=1.0)
        
        with tf.variable_scope('word_embedding', reuse=reuse):
            w = tf.get_variable('w', weight_dim, initializer=emb_weight_initializer)
            word_vecs = tf.nn.embedding_lookup(w, word_idx, name='word_vector')
        
        return word_vecs
    
    
    def _get_attention_layer(self, h, features, reuse):
        # z = beta * sum(alpha * annotation)
        beta_weight_dim = [self.hidden_dim, 1]
        beta_bias_dim = [1]
        alpha_weight_dim_1 = [self.hidden_dim, self.features_dim[1]]
        alpha_weight_dim_2 = [self.features_dim[1], 1]
        alpha_bias_dim_1 = self.features_dim[1]
        with tf.variable_scope('attention', reuse=reuse):
            with tf.variable_scope('beta'):
                w = tf.get_variable('w', beta_weight_dim, initializer=self.weight_initializer)
                b = tf.get_variable('b', beta_bias_dim, initializer=self.bias_initializer)
                _beta = tf.matmul(h, w) + b
                beta = tf.nn.sigmoid(_beta, name='beta')
            with tf.variable_scope('alpha'):
                we1 = tf.get_variable('w_e_1', alpha_weight_dim_1, initializer=self.weight_initializer)
                we2 = tf.get_variable('w_e_2', alpha_weight_dim_2, initializer=self.weight_initializer)
                be1 = tf.get_variable('b_e_1', alpha_bias_dim_1, initializer=self.bias_initializer)
                _he1 = tf.matmul(h, we1) + be1
                he1 = tf.nn.relu(_he1)
                _he2 = tf.matmul(he1, we2)
                he2 = tf.nn.relu(_he2)

                alpha = tf.nn.softmax(he2, name='alpha')

            # z is the context vector
            # alpha shape = [-1, 1] ---expand---> [-1, 1, 1]
            alpha = tf.expand_dims(alpha, axis=2)
            context = tf.reduce_sum(features * alpha, axis=1, name='context')
            context = tf.multiply(beta, context)

        return context, alpha, beta
        
    def _get_logits(self, h, reuse):
        logits_weight_dim_1 = [self.hidden_dim, self.wordvec_emb_dim]
        logits_weight_dim_2 = [self.wordvec_emb_dim, self.vocab_size]
        logits_bias_dim_1 = [self.wordvec_emb_dim]
        logits_bias_dim_2 = [self.vocab_size]
        with tf.variable_scope('logits', reuse=reuse):
            w_1 = tf.get_variable('w_1', logits_weight_dim_1, initializer=self.weight_initializer)
            w_2 = tf.get_variable('w_2', logits_weight_dim_2, initializer=self.weight_initializer)
            b_1 = tf.get_variable('b_1', logits_bias_dim_1, initializer=self.bias_initializer)
            b_2 = tf.get_variable('b_2', logits_bias_dim_2, initializer=self.bias_initializer)
            
            _logits = tf.matmul(h, w_1) + b_1
            _logits = tf.nn.tanh(_logits)
            logits = tf.matmul(_logits, w_2) + b_2
            
        return logits
            
        
    def _get_loss(self, 
                  logits, labels_gt, 
                  alphas, reg_coef,
                  reuse):
        # loss = cross entropy (NLL) + regularization
        with tf.variable_scope('loss', reuse=reuse):
            ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=labels_gt, name='ce')
            nll = tf.reduce_sum(ce, name='nll')
            alphas_total = tf.reduce_sum(alphas, name='alpha_total')
            reg = reg_coef * tf.reduce_sum((16./196 - alphas_total) ** 2, name='reg_loss')
            
            loss = nll + reg
            
        return loss
    
    def sample(self, num_samples):
        features = self.features
        c,h = self._get_initial_lstm(self.features)
        state = tf.contrib.rnn.LSTMStateTuple(c, h)
        
        alphas = []
        betas = []
        sampled_words = []
        
        LSTM = tf.contrib.rnn.LayerNormBasicLSTMCell(self.hidden_dim,
                                                     dropout_keep_prob=0.5,
                                                     dropout_prob_seed=0)
        for tstep in range(self.n_time_steps):
            if tstep == 0:
                # append with <start>
                # remember to set dropout to test time mode
                word_embs = self._word_embedding(inputs=tf.fill([tf.shape(features)[0]], self._start))
            context, alpha, beta = self._get_attention_layer(h, features, reuse=(tstep is not 0))
            # context = [n_features, features_dim[1]]
            # alpha = [n_features, 1, 1]
            # beta = [n_features, 1]
            alphas.append(alpha)
            betas.append(beta)
            
            lstm_input_word = tf.concat([word_embs[:, tstep, :], context], axis=1)
            
            # run the LSTM    
            with tf.variable_scope('lstm', reuse=(tstep is not 0)):
                # each LSTM gate takes input [h_last, context, Ey]
                # the dimension of word vectors = [n_examples, n_words_in_example, word_emb_dim]
                # each step we feed in a word in a sentence
                output, state = LSTM(lstm_input_word, state=state)
                [c, h] = [state.c, state.h]
                
            # LSTM creates an output at each step, hence the loss
            logits = self._get_logits(h, reuse=(tstep is not 0))

            loss += self._get_loss(logits=logits, 
                                   labels_gt=captions_clipped[:,tstep], 
                                   alphas=alphas, 
                                   reg_coef=self.loss_reg_coef,
                                   reuse=(tstep is not 0))

        return loss / tf.to_float(batch_size)
        
        
        
    

In [195]:
from utils import load_coco_data

In [ ]:
data = load_coco_data(data_dir=, split='test')
word_to_idx = data['word_to_idx']
# load val dataset to print out bleu scores every epoch
val_data = load_coco_data(data_dir='./data', split='val')


solver = CaptioningSolver(model, data, val_data, n_epochs=20, batch_size=128, update_rule='adam',
                                          learning_rate=0.001, print_every=1000, save_every=1, image_path='./image/',
                                    pretrained_model=None, model_path='model/lstm/', test_model='model/lstm/model-10',
                                     print_bleu=True, log_path='log/')

solver.train()

## TRAIN

In [18]:
model_save_path = '/root/DLResearch/show-attend-and-tell/model/lstm'
log_save_path = '/root/DLResearch/show-attend-and-tell/log'


In [33]:
#model = CaptionGenerator()
def train(model, data, val_data,
          log_path, model_path,
          n_epochs=10, batch_size=64, 
          print_every=100, save_every=1):
    
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    if not os.path.exists(log_path):
        os.makedirs(log_path)
        
    # build model
    loss = model.build_model()
    tf.get_variable_scope().reuse()
    _, _, generated_captions = self.model.build_model(max_len=20)

    # get loss
    global_step = tf.Variable(0 ,trainable=False, name='global_step')
    optimizer = tf.train.AdamOptimizer()
    grads_and_vars = optimizer.compute_gradients()
    train_op = optimizer.apply_gradients(grads_and_vars, global_step)

    # log
    tf.summary.scalar('batch_loss', loss)
    # [tf.summary.histogram(var.op.name, var) for var in tf.trainable_variables()]
    [tf.summary.histogram(var.op.name + '/gradients', grad) for var,loss in grads_and_vars]
    summary_op = tf.summary.merge_all()


    print ("The number of epoch: %d" %self.n_epochs)
    print ("Data size: %d" %n_examples)
    print ("Batch size: %d" %self.batch_size)
    print ("Iterations per epoch: %d" %n_iters_per_epoch)

    # train
    onfig = tf.ConfigProto(allow_soft_placement = True)
    config.gpu_options.allow_growth = True
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        summary_writer = tf.summary.FileWriter(logdir=log_save_path, graph=tf.get_default_graph())

        prev_epoch_loss = -1
        batches_per_epoch = data['captions'].shape[0] // batch_size + 1
        iters_val_per_epoch = val_features.shape[0] // self.batch_size + 1
        for epoch in n_epochs:
            current_epoch_loss = 0
            epoch_start_time = time.time()
            for batch in range(batches_per_epoch):
                # get the feed_dict
                start_idx = batch * batch_size
                end_idx = (batch+1) * batch_size
                captions_batch = captions[start_idx:end_idx]
                image_idxs_batch = image_idxs[start_idx:end_idx]
                features_batch = features[start_idx:end_idx]
                feed_dict = {model.features: features_batch,
                            model.captions: captions_batch}
                # run it
                _, batch_loss = sess.run([train_op, loss], feed_dict=feed_dict)
                current_epoch_loss += batch_loss
                
                # log it
                if batch % 20 == 0:
                    summary = sess.run(summary_op, feed_dict=feed_dict)
                    summary_writer.add_summary(summary, global_step)
                if (i+1) % print_every == 0:
                    print ("\nTrain loss at epoch %d & iteration %d (mini-batch): %.5f" %
                           (epoch, batch, batch_loss))
                    captions_gt = captions[1:10]
                    captions_g = sess.run(generated_captions)
            
            print ("Previous epoch loss: ", prev_epoch_loss)
            print ("Current epoch loss: ", current_epoch_loss)
            print ("Elapsed time: ", time.time() - epoch_start_time)
            prev_epoch_loss = current_epoch_loss
                
            # copied from original implementation
            all_gen_cap = np.ndarray((val_features.shape[0], 20))
            for batch in range(iters_val_per_epoch):
                start_idx = batch * batch_size
                end_idx = (batch+1) * batch_size
                features_batch = val_features[start_idx:end_idx]
                feed_dict = {self.model.features: features_batch}
                gen_cap = sess.run(generated_captions, feed_dict=feed_dict)  
                all_gen_cap[start_idx:end_idx] = gen_cap

            all_decoded = decode_captions(all_gen_cap, self.model.idx_to_word)
            if not os.path.exists('./tmp/val/'):
                os.makedirs('./tmp/val/')
            save_pickle(all_decoded, "./tmp/val/val.candidate.captions.pkl")
            scores = evaluate(data_path='./data', split='val', get_scores=True)
            write_bleu(scores=scores, path=model_path, epoch=epoch)
            
            # save model
            if (epoch+1) % save_every == 0:
                saver.save(sess, os.path.join(model_path, 'model,ckpt'), global_step=global_step)

### TESTING THE CODE!!!

In [5]:
import pickle
test_data = {}
with open('../show-attend-and-tell/__train.features.pkl', 'rb') as f:
    test_data['features'] = pickle.load(f, encoding='latin1')
with open('../show-attend-and-tell/data/train/train.captions.pkl', 'rb') as f:
    test_captions = pickle.load(f, encoding='latin1')
    # because the test_features are picked as [21, :]
    test_data['captions'] = test_captions[:21,:]
with open('../show-attend-and-tell/data/train/word_to_idx.pkl', 'rb') as f:
    test_data['word_to_idx'] = pickle.load(f, encoding='latin1')
with open('../show-attend-and-tell/data/train/train.file.names.pkl', 'rb') as f:
    test_data['file_names'] = pickle.load(f, encoding='latin1')
with open('../show-attend-and-tell/data/train/train.image.idxs.pkl', 'rb') as f:
    test_data['image_idxs'] = pickle.load(f, encoding='latin1')

In [208]:
print(test_data['features'].shape)
print(test_data['captions'].shape)
print(len(test_data['word_to_idx']))
print(test_data['file_names'].shape)
print(test_data['image_idxs'].shape)

(21, 196, 512)
(21, 17)
23110
(82783,)
(399998,)
